In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import plotly.express as px

In [2]:
color_dict = {
    'blue': 7, 
    'light gray': 9, 
    'green': 6, 
    'bright green': 36, 
    'lime': 34, 
    'yellow': 3, 
    'orange': 4, 
}

brick_types = ['1x1_3024', '1x2_3023', '2x4_3020', '2x2_3022']

shopping_list = pd.read_csv('results/brick_count_all.csv')
shopping_list.drop(columns=['layer_name', 'total', 'Unnamed: 0'], inplace=True)
shopping_list = shopping_list.groupby(by='layer_color').sum().reset_index()
shopping_list['color_code'] = shopping_list.layer_color.map(lambda x: color_dict[x])
for col in brick_types: 
    shopping_list[col] = shopping_list[col].astype(int)

brick_names_all = []
for color in shopping_list.layer_color: 
    for brickType in ['1x1_3024', '1x2_3023', '2x4_3020', '2x2_3022']: 
        if shopping_list[shopping_list.layer_color == color][brickType].iloc[0] > 0: 
            size = brickType[:3]
            size = ' '.join(size)
            brick_name = f'{color.title()} Plate {size}'
            brick_names_all.append(brick_name)
            
shopping_list

,layer_color,1x1_3024,1x2_3023,2x4_3020,2x2_3022,color_code
0,blue,800,206,149,90,7
1,bright green,210,43,44,39,36
2,green,501,135,304,77,6
3,light gray,692,159,427,94,9
4,lime,64,19,4,6,34
5,orange,238,54,16,20,4
6,yellow,16,4,0,1,3


In [3]:
def construct_url(color, part_number, part_name, min_qty):
    BASE_URL = "https://www.bricklink.com/v2/catalog/catalogitem.page"
    params = {
        'P': part_number,
        'name': f"Plate%20{'%20'.join(part_name.split())}",
        'category': '[Plate]',
        'T': 'S',
        'C': f'{color}',
        'O': f'{{"color":"{color}","minqty":"{min_qty}","reg":"-1","rpp":"500","iconly":0}}'
    }
    return BASE_URL + "?" + "&".join([f"{k}={v}" for k, v in params.items()])

def extract_value_from_text(text):
    matches = re.findall(r'\d+\.\d+', text)
    return float(matches[0]) if matches else None

def get_vendors_from_soup(soup, min_qty, browser, color, brick_number, brick_name, attempt=1, max_attempts=3):
    items = soup.find_all('tr', class_='pciItemContents')
    vendors = []
    placeholder = "[%strColorString%]"

    for item in items:
        vendor_name = item.find('span', class_='pspStoreName').text
        brick_description = item.find('a', class_='pciItemNameLink').text
        price_text = item.find('td', style="text-align: right;", width="90px").text.strip()
        min_buy_section = item.find('td', width='120px').find_all('span', style='font-size: 11px;')
        min_buy_text = next((span.text.replace("Min Buy:", "").strip() for span in min_buy_section if "Min Buy:" in span.text), None)
        shop_link = "https://www.bricklink.com" + item.find('a')['href']

        if placeholder in brick_description and attempt < max_attempts:
            # Reload the page and retry
            print(f"Placeholder detected, retrying... Attempt {attempt}/{max_attempts}")
            time.sleep(2)  # Wait before retrying
            url = construct_url(color, brick_number, brick_name, min_qty)
            browser.get(url)
            browser.implicitly_wait(10)
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            return get_vendors_from_soup(soup, min_qty, browser, color, brick_number, brick_name, attempt + 1, max_attempts)
        elif placeholder in brick_description:
            # Use a default value if max attempts reached
            brick_description = f"Default Color Plate {brick_name}"

        vendors.append({
            'vendor_name': vendor_name,
            'brick_description': brick_description,
            'price': extract_value_from_text(price_text),
            'minimum_buy': extract_value_from_text(min_buy_text) if min_buy_text else None,
            'shop_link': shop_link,
            'num_bricks': min_qty
        })

    return vendors

In [7]:
def get_vendors(row): 
    # get chrome driver for selenium  
    driver_path = 'chromedriver.exe'
    s = Service(driver_path)
    browser = webdriver.Chrome(service=s)

    # Define color and brick_types from the row
    color = row.color_code
    brick_types = ['1x1_3024', '1x2_3023', '2x4_3020', '2x2_3022']
    print(row.layer_color)

    vendors_list = []  # Store vendors from each brick type
    for brick_type in brick_types: 
        if row[brick_type] == 0: 
            print(f'{row.layer_color} {brick_type} is zero, skipping.')
            continue
        
        print(brick_type)
        
        brick_number = brick_type[-4:]
        brick_name = brick_type[:3]
        min_qty = row[brick_type]
        url = construct_url(color, brick_number, brick_name, min_qty)
        
        browser.get(url)
        browser.implicitly_wait(10)
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        vendors = get_vendors_from_soup(soup, min_qty, browser, color, brick_number, brick_name)
        vendors_list.extend(vendors)  # Append vendors from current brick type

    browser.quit()
    return pd.DataFrame(vendors_list)

results_df = pd.concat(shopping_list.apply(lambda row: get_vendors(row), axis=1).tolist(), ignore_index=True)
results_df.to_csv('results/brick_vendors.csv')

blue
1x1_3024
1x2_3023
2x4_3020
2x2_3022
bright green
1x1_3024
1x2_3023
2x4_3020
2x2_3022
green
1x1_3024
1x2_3023
2x4_3020
2x2_3022
light gray
1x1_3024
1x2_3023
2x4_3020
2x2_3022
lime
1x1_3024
1x2_3023
2x4_3020
2x2_3022
orange
1x1_3024
1x2_3023
2x4_3020
2x2_3022
yellow
1x1_3024
1x2_3023
yellow 2x4_3020 is zero, skipping.
2x2_3022


In [17]:
results_df.sort_values(by='num_bricks', ascending=False)

,vendor_name,brick_description,price,minimum_buy,shop_link,num_bricks
0,Wunschbox 15 bis 30 % off,Blue Plate 1 x 1,0.01,20.00,https://www.bricklink.com//store.bricklink.com...,800
21,brickazon,Blue Plate 1 x 1,0.05,0.23,https://www.bricklink.com//store.bricklink.com...,800
23,Generation Bricks,Blue Plate 1 x 1,0.05,5.00,https://www.bricklink.com//store.bricklink.com...,800
24,wichtel10000 - SALE OUT !!,Blue Plate 1 x 1,0.05,1.00,https://www.bricklink.com//store.bricklink.com...,800
25,TIMBY'S CELLAR FREE📦📫*,Blue Plate 1 x 1,0.05,NaN,https://www.bricklink.com//store.bricklink.com...,800
...,...,...,...,...,...,...
7255,SV Brick Shop,Yellow Plate 2 x 2,0.02,1.00,https://www.bricklink.com//store.bricklink.com...,1
7254,Mathiesens Surplus (SALE!),Yellow Plate 2 x 2,0.02,134.08,https://www.bricklink.com//store.bricklink.com...,1
7253,Suschka_Brick,Yellow Plate 2 x 2,0.02,NaN,https://www.bricklink.com//store.bricklink.com...,1
7252,Teddy_Sell_Bricks [OLX],Yellow Plate 2 x 2,0.02,2.31,https://www.bricklink.com//store.bricklink.com...,1


In [8]:
results_df[results_df.brick_description == '[%strColorString%]Plate 1 x 2']

,vendor_name,brick_description,price,minimum_buy,shop_link,num_bricks


In [9]:
results_df.brick_description.unique()

array(['Blue Plate 1 x 1', 'Blue Plate 1 x 2', 'Blue Plate 2 x 4',
       'Blue Plate 2 x 2', 'Bright Green Plate 1 x 1',
       'Bright Green Plate 1 x 2', 'Bright Green Plate 2 x 4',
       'Bright Green Plate 2 x 2', 'Green Plate 1 x 1',
       'Green Plate 1 x 2', 'Green Plate 2 x 4', 'Green Plate 2 x 2',
       'Light Gray Plate 1 x 1', 'Light Gray Plate 1 x 2',
       'Light Gray Plate 2 x 4', 'Light Gray Plate 2 x 2',
       'Lime Plate 1 x 1', 'Lime Plate 1 x 2', 'Lime Plate 2 x 4',
       'Lime Plate 2 x 2', 'Orange Plate 1 x 1', 'Orange Plate 1 x 2',
       'Orange Plate 2 x 4', 'Orange Plate 2 x 2', 'Yellow Plate 1 x 1',
       'Yellow Plate 1 x 2', 'Yellow Plate 2 x 2'], dtype=object)

In [20]:
def greedy_set_cover(universe, subsets):
    """ Find a family of subsets that covers the universal set """
    universe = set(universe)
    covered = set()
    cover = []

    # While not all elements are covered
    while covered != universe:
        # Choose the subset that adds the most uncovered elements
        subset = max(subsets, key=lambda s: len(s - covered))
        cover.append(subset)
        covered |= subset

    return cover

def calc_total_cost(max_price): 

    df = results_df[results_df.price <= max_price]
    vendors_dict = df.groupby('vendor_name')['brick_description'].apply(set).to_dict()
    vendors_dict = dict(sorted(vendors_dict.items(), key=lambda item: len(item[1]), reverse=True))
    all_bricks = df['brick_description'].unique()
    
    minimal_vendors_sets = greedy_set_cover(all_bricks, vendors_dict.values())
    minimal_vendor_names = [vendor for vendor, bricks in vendors_dict.items() if bricks in minimal_vendors_sets]
    
    final_list = results_df[results_df.vendor_name.isin(minimal_vendor_names)]
    final_list = final_list.sort_values(by=['brick_description', 'price'])
    final_list = final_list.drop_duplicates(subset='brick_description', keep='first')
    final_list['cost'] = final_list.price * final_list.num_bricks
    
    total_cost_df = final_list[['vendor_name', 'num_bricks', 'cost']].groupby(by='vendor_name').sum().reset_index()
    total_cost_df['total_cost'] = total_cost_df.cost + 10
    print(f'\nmax price: {max_price} -- total cost: {total_cost_df.total_cost.sum()}')

    return total_cost_df.total_cost.sum(), final_list

def check_final_list_df(final_list_df): 
    missing_bricks = []
    for brick_name in brick_names_all: 
        if brick_name not in final_list_df.brick_description.unique():
            missing_bricks.append(brick_name)
    if missing_bricks: 
        print(f'missing bricks: {missing_bricks}')
    return missing_bricks

In [21]:
results_df = pd.read_csv('results/brick_vendors.csv')

res_dict = {}
price_options = {}
for max_price in range(1, 15): 
    max_price = max_price / 100
    total_cost, final_list_df = calc_total_cost(max_price)
    missing_bricks = check_final_list_df(final_list_df)
    if not missing_bricks: 
        price_options[max_price] = total_cost
    res_dict[max_price] = total_cost

price_options


max price: 0.01 -- total cost: 156.44000000000003
missing bricks: ['Bright Green Plate 1 x 2', 'Bright Green Plate 2 x 2', 'Green Plate 1 x 2', 'Green Plate 2 x 4', 'Green Plate 2 x 2', 'Light Gray Plate 1 x 1', 'Light Gray Plate 2 x 4', 'Lime Plate 1 x 1']

max price: 0.02 -- total cost: 168.01
missing bricks: ['Bright Green Plate 2 x 2', 'Green Plate 2 x 4', 'Light Gray Plate 1 x 1']

max price: 0.03 -- total cost: 176.71999999999997
missing bricks: ['Green Plate 2 x 4', 'Light Gray Plate 1 x 1', 'Light Gray Plate 2 x 4']

max price: 0.04 -- total cost: 313.03
missing bricks: ['Bright Green Plate 2 x 2']

max price: 0.05 -- total cost: 181.95
missing bricks: ['Light Gray Plate 1 x 1']

max price: 0.06 -- total cost: 330.88

max price: 0.07 -- total cost: 330.88

max price: 0.08 -- total cost: 330.88

max price: 0.09 -- total cost: 315.48

max price: 0.1 -- total cost: 315.48

max price: 0.11 -- total cost: 315.48

max price: 0.12 -- total cost: 316.21999999999997

max price: 0.13 --

{0.06: 330.88,
 0.07: 330.88,
 0.08: 330.88,
 0.09: 315.48,
 0.1: 315.48,
 0.11: 315.48,
 0.12: 316.21999999999997,
 0.13: 319.36,
 0.14: 319.36}

In [23]:
total_cost, final_list = calc_total_cost(0.05)
final_list[['vendor_name', 'brick_description', 'num_bricks', 'price']].to_csv('results/shopping_list_all.csv', index=False)
final_list.sort_values(by='brick_description') # .vendor_name.unique()


max price: 0.05 -- total cost: 181.95


,Unnamed: 0,vendor_name,brick_description,price,minimum_buy,shop_link,num_bricks,cost
8,8,Brick Takeover,Blue Plate 1 x 1,0.04,7.0,https://www.bricklink.com//store.bricklink.com...,800,32.00
255,255,Nordic Brix,Blue Plate 1 x 2,0.04,20.0,https://www.bricklink.com//store.bricklink.com...,206,8.24
937,937,Nordic Brix,Blue Plate 2 x 2,0.04,20.0,https://www.bricklink.com//store.bricklink.com...,90,3.60
569,569,Nordic Brix,Blue Plate 2 x 4,0.04,20.0,https://www.bricklink.com//store.bricklink.com...,149,5.96
1196,1196,Brick Takeover,Bright Green Plate 1 x 1,0.05,7.0,https://www.bricklink.com//store.bricklink.com...,210,10.50
1238,1238,Magic Magnus 🛒,Bright Green Plate 1 x 2,0.04,NaN,https://www.bricklink.com//store.bricklink.com...,43,1.72
1615,1615,Brick Takeover,Bright Green Plate 2 x 2,0.24,7.0,https://www.bricklink.com//store.bricklink.com...,39,9.36
1488,1488,Magic Magnus 🛒,Bright Green Plate 2 x 4,0.05,NaN,https://www.bricklink.com//store.bricklink.com...,44,2.20
1706,1706,Nordic Brix,Green Plate 1 x 1,0.03,20.0,https://www.bricklink.com//store.bricklink.com...,501,15.03
1767,1767,Nordic Brix,Green Plate 1 x 2,0.03,20.0,https://www.bricklink.com//store.bricklink.com...,135,4.05
